In [1]:
import pickle
import sys
import cv2
import time
import numpy as np
import ffmpeg
from pathlib import Path

from moviepy.editor import VideoFileClip
from IPython.display import HTML

repo_path = Path(".").absolute().parent
data_path = Path(".") / "videos"
    
sys.path.append(str(repo_path))

from lanedetection import BirdsEye, LaneFilter, Curves, roi, get_points, resize_image

In [2]:
class PipeLine:
    def __init__(self, params):
        r"""
        width: width of image
        height: height of image
        cali_path: calibration path (should be preprocessed by chessboard.py)
        src_ratio: points to detect line (with ratio from 0~1) 
        dest_ratio: points to transform (with ratio from 0~1)
        lanefilter: params dictionary for LaneFilter
        curves: params dictionary for Curves
        """
        self.w, self.h = params["width"], params["height"]
        self.calibration_data = pickle.load(open(params["cali_path"], "rb"))
        self.src_ratio = params["src_ratio"]
        self.dest_ratio = params["dest_ratio"]

        matrix = self.calibration_data['camera_matrix']
        distortion_coef = self.calibration_data['distortion_coefficient']
        source_points = get_points(self.src_ratio, self.w, self.h)
        dest_points = get_points(self.dest_ratio, self.w, self.h)
        
        self.birdseye = BirdsEye(source_points, dest_points, matrix, distortion_coef)
        self.lanefilter = LaneFilter(params['lanefilter'])
        self.curves = Curves(params['curves'])
        
    def process(self, frame):
        frame = resize_image(frame, self.w, self.h)
        ground_img = self.birdseye.undistort(frame)
        binary = self.lanefilter.apply(ground_img)
        bird_mask = np.logical_and(self.birdseye.sky_view(binary), roi(binary)).astype(np.uint8)
        bird_ground_img = self.birdseye.sky_view(ground_img)
        bird_img = cv2.bitwise_and(bird_ground_img, bird_ground_img, mask=bird_mask)

        result = self.curves.fit(bird_mask, bird_img)

        ground_img_with_projection = self.birdseye.project(ground_img, binary, 
                                                    result['pixel_left_best_fit_curve'], 
                                                    result['pixel_right_best_fit_curve'], 
                                                    result['left_color'][1], 
                                                    result['right_color'][1])
        text_pos = f"vehicle position: {result['vehicle_position_words']}"
        text_l = f"left radius: {np.round(result['left_radius'], 2)} | color: {result['left_color'][0]}"
        text_r = f"right radius: {np.round(result['right_radius'], 2)} | color {result['right_color'][0]}"  
        text_color = (255, 255, 255)
        cv2.putText(ground_img_with_projection, text_l, (20, 40), 
                    cv2.FONT_HERSHEY_PLAIN, 1, text_color, 2)
        cv2.putText(ground_img_with_projection, text_r, (20, 60), 
                    cv2.FONT_HERSHEY_PLAIN, 1, text_color, 2)
        cv2.putText(ground_img_with_projection, text_pos, (20, 80), 
                    cv2.FONT_HERSHEY_PLAIN, 1, text_color, 2)
        
        return ground_img_with_projection

In [9]:
params = {
    'width': 640,
    'height': 480,
    'cali_path': "calibration_data.p",
    'src_ratio': [(0.35, 0.55), (0.0, 1.0), (0.70, 0.55), (0.9, 1.0)],
    'dest_ratio': [(0.25, 0.0), (0.25, 1.0), (0.75, 0.0), (0.75, 1.0)],
    # to LaneFilter
    'lanefilter' :{
        'sat_thresh': 120,
        'light_thresh': 40,
        'light_thresh_agr': 205,
        'grad_thresh': (0.7, 1.4),
        'mag_thresh': 40,
        'x_thresh': 20
    },
    # to curves
    'curves': {
        'number_of_windows': 9,
        'margin': 50,
        'minimum_pixels': 50,
        'ym_per_pix': 30 / 720,
        'xm_per_pix': 3.7 / 700
    }
}

pipeline = PipeLine(params)

In [10]:
video_path = str(data_path / "sample_video1.mp4")
output_path = str(data_path / "video_output.mp4")
clip1 = VideoFileClip(video_path);
white_clip = clip1.fl_image(pipeline.process) 
%time white_clip.write_videofile(output_path, audio = False);

t:   0%|                                                                             | 0/618 [00:00<?, ?it/s, now=None]

Moviepy - Building video videos\video_output.mp4.
Moviepy - Writing video videos\video_output.mp4



Moviepy - Done !
Moviepy - video ready videos\video_output.mp4
Wall time: 42.8 s


In [11]:
HTML("""
<video width="640" height="480" controls>
  <source src="{0}">
</video>
""".format(output_path))